In [40]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import shap
from sklearn.metrics.pairwise import cosine_similarity
import copy
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt


In [3]:
np.random.seed(0)

In [4]:
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
class_names = ['atheism', 'christian']

In [5]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False, token_pattern=r"\b\w+\b")
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

In [6]:
lreg = LogisticRegression(random_state=0, solver='lbfgs')
lreg.fit(train_vectors, newsgroups_train.target)

nbayes = MultinomialNB()
nbayes.fit(train_vectors, newsgroups_train.target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [7]:
pred_lreg = lreg.predict(test_vectors)
print(sklearn.metrics.f1_score(newsgroups_test.target, pred_lreg, average='binary'))

pred_nbayes = nbayes.predict(test_vectors)
print(sklearn.metrics.f1_score(newsgroups_test.target, pred_nbayes, average='binary'))

0.9205607476635513
0.8816964285714286


### Global weights

In [10]:
global_nbayes_params = nbayes.coef_[0]
global_lreg_params = lreg.coef_[0]

In [11]:
selected_params_nbayes = np.abs(global_nbayes_params).argsort()[-10:][::-1]
selected_params_nbayes_values = global_nbayes_params[selected_params_nbayes]

selected_params_lreg = np.abs(global_lreg_params).argsort()[-10:][::-1]
selected_params_lreg_values = global_lreg_params[selected_params_lreg]

In [12]:
selected_feature_names_nbayes = []
selected_feature_names_lreg = []
for sel_p_val in selected_params_nbayes:
    selected_feature_names_nbayes.append(list(vectorizer.vocabulary_.keys())[list(vectorizer.vocabulary_.values()).index(sel_p_val)])  # Prints george
    
selected_feature_names_lreg = []
for sel_p_val in selected_params_lreg:
    selected_feature_names_lreg.append(list(vectorizer.vocabulary_.keys())[list(vectorizer.vocabulary_.values()).index(sel_p_val)])  # Prints george

### Local Naive Bayes weights

In [21]:
idx = 83
local_gt = train_vectors[idx].nonzero()[1]

In [121]:
np.random.randint(0, train_vectors.shape[0], 10)

array([ 893,  304,  785,  706,  979,  709,  743,  728, 1027,  566])

In [117]:
local_weights_nbayes = np.abs(global_nbayes_params[local_gt]).argsort()[-10:][::-1]
local_weights_lreg = np.abs(global_lreg_params[local_gt]).argsort()[-10:][::-1]

selected_local_feature_names_nbayes = []
selected_local_feature_names_lreg = []

for sel_p_val in local_weights_nbayes:
    selected_local_feature_names_nbayes.append(list(vectorizer.vocabulary_.keys())[list(vectorizer.vocabulary_.values()).index(sel_p_val)])  # Prints george

for sel_p_val in local_weights_lreg:
    selected_local_feature_names_lreg.append(list(vectorizer.vocabulary_.keys())[list(vectorizer.vocabulary_.values()).index(sel_p_val)])  # Prints george

### LIME weights

In [14]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

from lime import lime_text
from sklearn.pipeline import make_pipeline
c_lreg = make_pipeline(vectorizer, lreg)
c_nbayes = make_pipeline(vectorizer, nbayes)

In [23]:
explained_class = 1
exp_lreg = explainer.explain_instance(newsgroups_train.data[idx], c_lreg.predict_proba, num_features=train_vectors.shape[1], labels=(explained_class, ))
exp_lreg_top_10 = explainer.explain_instance(newsgroups_train.data[idx], c_lreg.predict_proba, num_features=10, labels=(explained_class, ))

#print('Document id: %d' % idx)
#print('Probability(christian) =', c_lreg.predict_proba([newsgroups_train.data[idx]])[0,1])
#print('True class: %s' % class_names[newsgroups_train.target[idx]])

split() requires a non-empty pattern match.
split() requires a non-empty pattern match.


In [24]:
explained_class = 1
exp_nbayes = explainer.explain_instance(newsgroups_train.data[idx], c_nbayes.predict_proba, num_features=train_vectors.shape[1], labels=(explained_class, ))
exp_nbayes_top_10 = explainer.explain_instance(newsgroups_train.data[idx], c_nbayes.predict_proba, num_features=10, labels=(explained_class, ))

#print('Document id: %d' % idx)
#print('Probability(christian) =', c_nbayes.predict_proba([newsgroups_train.data[idx]])[0,1])
#print('True class: %s' % class_names[newsgroups_train.target[idx]])

split() requires a non-empty pattern match.
split() requires a non-empty pattern match.


In [25]:
selected_features_lime_nbayes_top_10 = []
selected_features_lime_lreg_top_10 = []

for e in exp_lreg_top_10.as_list():
    selected_features_lime_lreg_top_10.append(vectorizer.vocabulary_[e[0]])

for e in exp_nbayes_top_10.as_list():
    selected_features_lime_nbayes_top_10.append(vectorizer.vocabulary_[e[0]])

In [26]:
selected_features_lime_nbayes = np.zeros((1, train_vectors.shape[1]))
selected_features_lime_lreg = np.zeros((1, train_vectors.shape[1]))

for e in exp_lreg.as_list():
    selected_features_lime_lreg[:, vectorizer.vocabulary_[e[0]]] = e[1]

for e in exp_nbayes.as_list():
    selected_features_lime_nbayes[:, vectorizer.vocabulary_[e[0]]] = e[1]

### KernelShap Weights

In [27]:
median_train = np.median(train_vectors.toarray(), axis=0).reshape(1, -1)

In [65]:
lreg_lambda = lambda x: lreg.predict_proba(x)[:, explained_class]
nbayes_lambda = lambda x: nbayes.predict_proba(x)[:, explained_class]

shap_explainer_nbayes = shap.KernelExplainer(nbayes_lambda, median_train)
shap_values_nbayes = shap_explainer_nbayes.shap_values(train_vectors[idx], nsamples=1000)
shap_values_nbayes_top_10 = np.abs(shap_values_nbayes.flatten()).argsort()[-10:][::-1]

shap_explainer_lreg = shap.KernelExplainer(lreg_lambda, median_train)
shap_values_lreg = shap_explainer_lreg.shap_values(train_vectors[idx], nsamples=1000)
shap_values_lreg_top_10 = np.abs(shap_values_lreg.flatten()).argsort()[-10:][::-1]

future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.


future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.


In [59]:
cosine_similarity(shap_values_lreg, selected_features_lime_lreg)

array([[0.86839026]])

### Evaluation

In [31]:
explicand = copy.copy(test_vectors[idx].toarray())

In [53]:
def predict_replacement(explicand, x_train, selected_features, model, explained_class):
    result = 0
    for j in selected_features:
        explicand_copy = copy.copy(explicand)
        explicand_f_val = explicand_copy[:, j][0]
        feature_values = train_vectors[:, j].toarray().flatten()
        bin_count, bin_edge = np.histogram(feature_values, bins=4)
        
        for i in range(0, len(bin_edge) - 1):
            if explicand_f_val >= bin_edge[i] and  explicand_f_val < bin_edge[i+1]:
                bin_idx = i
                bin_avg = np.mean([bin_edge[i], bin_edge[i+1]])

        prior = bin_count[bin_idx] / len(feature_values)
        explicand_copy[:, j] = bin_avg
        
        new_pred = model.predict_proba(explicand_copy)[0][explained_class]
        
        result += new_pred * prior
    
    return result

In [48]:
base_pred_lreg = lreg.predict_proba(explicand)[0][explained_class]
base_pred_nbayes = nbayes.predict_proba(explicand)[0][explained_class]

In [118]:
new_pred_method_1_lreg_local = predict_replacement(explicand, train_vectors, local_weights_lreg, lreg, explained_class)
new_pred_method_1_nbayes_local = predict_replacement(explicand, train_vectors, local_weights_nbayes, lreg, explained_class)

In [67]:
new_pred_method_1_lreg_lime = predict_replacement(explicand, train_vectors, selected_features_lime_lreg_top_10, lreg, explained_class)
new_pred_method_1_nbayes_lime = predict_replacement(explicand, train_vectors, selected_features_lime_nbayes_top_10, lreg, explained_class)
new_pred_method_1_lreg_shap = predict_replacement(explicand, train_vectors, shap_values_lreg_top_10, nbayes, explained_class)
new_pred_method_1_nbayes_shap = predict_replacement(explicand, train_vectors, shap_values_nbayes_top_10, nbayes, explained_class)

In [103]:
def calculate_change_output(_explicand, train, selected_features, model, explained_class):
    explicand_copy = copy.copy(_explicand)

    for i in range(0, len(selected_features)):
        explicand_copy[:, selected_features[i]] = np.mean(train[:, selected_features[i]]) 

    new_pred = model.predict_proba(explicand_copy)[0][explained_class]
    
    return new_pred

In [104]:
explicand_copy = copy.copy(explicand)

new_pred_method_2_lreg_lime = calculate_change_output(explicand_copy, train_vectors, selected_features_lime_lreg_top_10, lreg, explained_class)
new_pred_method_2_nbayes_lime = calculate_change_output(explicand_copy, train_vectors, selected_features_lime_nbayes_top_10, nbayes, explained_class)
new_pred_method_2_lreg_shap = calculate_change_output(explicand_copy, train_vectors, shap_values_lreg_top_10, lreg, explained_class)
new_pred_method_2_nbayes_shap = calculate_change_output(explicand_copy, train_vectors, shap_values_nbayes_top_10, nbayes, explained_class)

In [119]:
new_pred_method_2_lreg_local = calculate_change_output(explicand_copy, train_vectors, local_weights_lreg, lreg, explained_class)
new_pred_method_2_nbayes_local = calculate_change_output(explicand_copy, train_vectors, local_weights_nbayes, nbayes, explained_class)

### Load explanations

In [12]:
import pickle

explanations_nbayes = pickle.load( open( "./comparison/explanations_nbayes.p", "rb" ) )
explanations_lreg = pickle.load( open( "./comparison/explanations_lreg.p", "rb" ) )

explanations_nbayes_top_10 = pickle.load( open( "./comparison/explanations_nbayes_top_10.p", "rb" ) )
explanations_lreg_top_10 = pickle.load( open( "./comparison/explanations_lreg_top_10.p", "rb" ) )

evaluation_lreg = pickle.load( open( "./comparison/evaluation_lreg.p", "rb" ) )
evaluation_nbayes = pickle.load( open( "./comparison/evaluation_nbayes.p", "rb" ) )

In [18]:
explanations_lreg

{83: {'lime': array([0., 0., 0., ..., 0., 0., 0.]),
  'shap': array([0., 0., 0., ..., 0., 0., 0.])}}